In this notebook we will deal with multiple destinations and drivers. Based on the given number of drivers we use K-means to cluster the destinations.  Then, within each driver's cluster, we create an algorithm that uses the A* algorithm to compute the best ordering of destinations, along with the optimal path from place to place.  

We use edge weights to allow for optimizing by travel time or total distance travelled. 

As with our work with one destination we will allow for further customization by the user - avoiding highways or providing a list of streets they would like to avoid.

NOTE:  the result of this notebook will be outputs that show the route, edge by edge, along with edge information.  Destination markers and turn directions have been added.  Further work will be done to simplify this output as was done with the one destination.

In [2]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import geopy.distance
from math import atan2, pi
from sklearn.cluster import KMeans

In [3]:
# Build a graph - this is the Whitby/Oshawa area for testing purposes
G = ox.graph_from_bbox(43.984503, 43.862879, -78.954552, -78.821660, network_type='drive')
# Add edge speed attributes
G = ox.add_edge_speeds(G)

In [4]:
# Check one edge 
G.edges[(699620, 4238293220, 0)]

{'osmid': 424417362,
 'lanes': '2',
 'name': 'Garrard Road',
 'highway': 'residential',
 'oneway': False,
 'length': 12.565,
 'speed_kph': 46.5}

In [5]:
# Check total number of edges 
len(G.edges)

12041

In [6]:
# We want to ensure every edge properly has a speed attribute 
good = 0
for edge in G.edges:
    if 'speed_kph' in G.edges[edge].keys():
        good += 1
    else:
        print(G.edges[edge]['highway'])
print(good)

12041


In [7]:
# We want to make sure all of the speeds are appropriate values
speeds = []
for edge in G.edges:
    speeds.append(G.edges[edge]['speed_kph'])

speeds = set(speeds)
print(speeds)

{64.4, 65.0, 100.0, 70.0, 71.0, 40.0, 75.0, 45.0, 46.5, 80.0, 49.6, 50.0, 20.0, 55.0, 55.2, 60.0, 30.0}


In [8]:
# Add travel time attributes
G = ox.add_edge_travel_times(G)

In [9]:
# Check one edge 
G.edges[(699620, 4238293220, 0)]

{'osmid': 424417362,
 'lanes': '2',
 'name': 'Garrard Road',
 'highway': 'residential',
 'oneway': False,
 'length': 12.565,
 'speed_kph': 46.5,
 'travel_time': 1.0}

In [10]:
# We want to ensure every edge properly has a travel time
good = 0
for edge in G.edges:
    if 'travel_time' in G.edges[edge].keys():
        good += 1
    else:
        print(G.edges[edge]['highway'])
print(good)

12041


In [11]:
# We want to make sure all of the travel times are appropriate values
times = []
for edge in G.edges:
    times.append(G.edges[edge]['travel_time'])

times = set(times)
print(min(times))
print(max(times))

0.3
143.6


In [12]:
def build_graph(east, west, south, north):
    """
    Define the limits of the map by specifying the boundary longitudes
    and latitudes.
    
    Create the graph object.
    
    Return the graph.
    """
    G = ox.graph_from_bbox(east, west, south, north, network_type = 'drive') 
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
    return G

### Helper Functions

In [13]:
def address_to_coord(address):
    """
    Take address information and use Geopy to get its coordinates.   
    """
    locator = Nominatim(user_agent = 'capstone2_project')
    location = locator.geocode(address)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        print('Address Not Found')

In [14]:
# Define starting point
start = address_to_coord('209 Dundas St E Whitby')
start

(43.879921100000004, -78.94055259302831)

In [15]:
# Define destinations
dest1 = address_to_coord('910 Dundas St W Whitby')
dest2 = address_to_coord('3100 Garden Street Whitby')
dest3 = address_to_coord('4081 Thickson Road N Whitby')
dest4 = address_to_coord('728 Anderson Street Whitby')
dest5 = address_to_coord('1801 Dundas St E Whitby')
dest6 = address_to_coord('3500 Brock St N Whitby')

In [16]:
destinations = [dest1, dest2, dest3, dest4, dest5, dest6]
destinations

[(43.8761031, -78.9642643),
 (43.90189732393074, -78.94123924628661),
 (43.92343540288797, -78.92817613088428),
 (43.8934292, -78.9266608),
 (43.8880942, -78.9052277),
 (43.91177930263158, -78.95641609868422)]

In [17]:
def cluster_destinations(destinations, drivers):
    """
    Use the list of destination coordinates and the number of drivers.
    Perform K-Means to cluster the destinations.
    Returns the list of destination tuples separated as sublists according
    to their cluster.
    """
    kmeans = KMeans(n_clusters=drivers)
    kmeans.fit(destinations)
    clusters = []
    for i in range(drivers):
        group = []
        for j in range(len(destinations)):
            if kmeans.labels_[j] == i:
                group.append(destinations[j])
        clusters.append(group)
    return clusters

In [19]:
clusters = cluster_destinations(destinations, 2)
clusters

[[(43.92343540288797, -78.92817613088428),
  (43.8934292, -78.9266608),
  (43.8880942, -78.9052277)],
 [(43.8761031, -78.9642643),
  (43.90189732393074, -78.94123924628661),
  (43.91177930263158, -78.95641609868422)]]

In [20]:
def coords_to_node(graph, coords):
    """
    Take coordinates and returns the closest node in the graph.
    """
    return ox.get_nearest_node(graph, coords)

In [21]:
start_node = coords_to_node(G, start)
start_node

392180675

In [29]:
# Get destination nodes
# Keep as a list of lists - according to clusters
destination_nodes = []
for i in range(len(clusters)):
    inodes = []
    for x in clusters[i]:
        inodes.append(coords_to_node(G, x))
    destination_nodes.append(inodes)
destination_nodes

[[3877460879, 392171046, 290584416], [392166620, 392164099, 392177756]]

In [30]:
def node_to_coords(graph, node):
    """
    Gets the latitude and logitude of a graph node.
    """
    return (graph.nodes[node]['y'], graph.nodes[node]['x'])

In [31]:
# We now work within a cluster 
# We want to keep track of the destinations visited within that cluster 
# and the optimal path from to visit each destination within the cluster.
visited = []
route = []

In [32]:
def aStarMulti(start_node, destinations_nodes, graph, weight, visited, route):
    """
    This algorithm will determine the order in which to visit each destination in each cluster,
    and the optimal path from the start location, through all destinations.
    We take the starting location, the list of destinations, the graph, the chosen weight - 
    'travel_time' or 'length' - and the two previous lists. 
    This is a recursive function.  While there are destinations still to visit, the reached
    destination becomes the starting point for the next function call. 
    """
   
    if not destinations_nodes:
        return 'Done'    
    # For each destination, calculate the distance to the starting point
    distances = []
    for d in destinations_nodes:
        distances.append(geopy.distance.distance(node_to_coords(graph, start_node), 
                                                     node_to_coords(graph, d)).m)
    # Get the index of the closest destination
    minIndex = distances.index(min(distances))
    # Get the corresponding destination node
    minNode = destinations_nodes[minIndex]
    # Add this to the list of visited destinations
    visited.append(minNode)
    # Get the optimal path to this destination
    path = nx.astar_path(graph, start_node, minNode, weight=weight)
    # Add this to the current route
    route.append(path)
    # Remove this destination from the list
    destinations_nodes.pop(minIndex)
    # If there are still destinations to visit, call the function again with the 
    # current location as the new starting point
    if destination_nodes:
        aStarMulti(minNode, destinations_nodes, graph, weight, visited, route)

In [33]:
# In our example we are working with two clusters 
# Get the route for the first cluster of destinations
aStarMulti(start_node, destination_nodes[0], G, 'travel_time', visited, route)

In [34]:
# Check that destinations have been visited
visited

[392171046, 290584416, 3877460879]

In [35]:
# Our current route is a list of lists, and since the end point of each leg becomes
# the starting point for the next one, these nodes appear twice. 
# Here we want to remove this duplication while also converting the route into one list
route_list = []
route_list.append(route[0][0])
# For each leg of the route
for i in route:
    # For each node in the leg
    for j in i:
        if route_list[-1] != j:
            route_list.append(j)

In [36]:
def direction_between_points(pt1, pt2):
    """
    Takes two tuples of latitudes and longitudes.
    Returns the compass direction from point 1 to point 2.
    """
    delta_x = pt2[1] - pt1[1]
    delta_y = pt2[0] - pt1[0]
    
    degrees = atan2(delta_x, delta_y) / pi*180
    
    if degrees < 0:
        degrees = 360 + degrees
    
    compass = ['north', 'northeast', 'east', 'southeast', 
               'south', 'southwest', 'west', 'northwest', 'north']
    
    compass_dir = round(degrees/45)
    return compass[compass_dir]

In [37]:
def full_route_info(graph, route, dests_list):
    """
    Creates a DataFrame containing the starting node, street name,
    length, direction of travel, speed, and ending node, for each edge in the graph. 
    We will also label which nodes represent the user's destinations. 
    """
    route_info = list()
    num_steps = len(route)
    i = 0
    j = 1
# We want to get the information for every step in the route
# We handle if any information is unlisted or unknown
    while j < num_steps:
        step_info = list()
        step_info.append(route[i])
        try:
            name = graph[route[i]][route[j]][0]['name']
            # Sometimes a street may change names within a segment - for example, switching
            # from west to east.  In these cases we will just take the first name.
            if type(name) == list:
                name = name[0]
        except KeyError:
            # Sometimes street names are labeled as "links"
            # In these cases I will relabel them with the previous street
            try:
                name = graph[route[i-1]][route[j-1]][0]['name']
            except KeyError:
                name = 'Unnamed'
        step_info.append(name)
        try:
            seg_len = graph[route[i]][route[j]][0]['length']
        except KeyError:
            seg_len = 0
        step_info.append(seg_len)
        # We want to calculate the direction the edge is travelling in
        start = node_to_coords(graph, route[i])
        end = node_to_coords(graph, route[j])
        # Use above function to calculate the direction from the start of the 
        # edge to its end
        step_info.append(direction_between_points(start, end))
        
        seg_speed = graph[route[i]][route[j]][0]['speed_kph']
        step_info.append(seg_speed)
        step_info.append(route[j])
        # Identify if the end node of the edge is a destination
        if route[j] in dests_list:
            step_info.append('Yes')
        else:
            step_info.append('No')
        
        route_info.append(step_info)
        i +=1
        j +=1

    # Create a data frame of this information
    columns = ['Start Node', 'Street Name', 'Length', 'Direction', 'Speed', 'End Node', 'Destination?']
    return pd.DataFrame(route_info, columns=columns)

In [38]:
full_route_df = full_route_info(G, route_list, visited)

In [39]:
def get_turn_dir(graph, node1, node2, node3):
    """
    node1 is starting node for first segment,
    node2 is the shared node between segments,
    node3 is the end node of second segment.
    """
    # First, get coordinates for each of the nodes
    p1 = [graph.nodes[node1]['x'], graph.nodes[node1]['y']]
    p2 = [graph.nodes[node2]['x'], graph.nodes[node2]['y']]
    p3 = [graph.nodes[node3]['x'], graph.nodes[node3]['y']]
    
    # Next, define the line segments as vectors
    p1top2 = [m - n for m,n in zip(p2,p1)]
    p2top3 = [m - n for m,n in zip(p3,p2)]
    p1top3 = [m - n for m,n in zip(p3,p1)]
    
    # Next, compute the cross product of the vectors
    crossprod = np.cross(p1top3, p1top2)
    
    if crossprod > 0:
        return 'Right'
    elif crossprod < 0:
        return 'Left'
    else:
        return 'No turn'

In [40]:
def add_turns(route_df, graph):
    """
    Locate all the rows where the street name changes in the following row.
    Create a dictionary with these as keys - the values will calculate the direction
    of the turn using the previous function. 
    Adds a new column in the df showing the turn direction.
    """
    # Get the rows where the street changes in the following row
    # These will be the turns
    turn_indexes = []
    for ind in route_df.index:
        if ind == route_df.index[-1]:
            pass
        elif route_df.iloc[ind]['Street Name'] == route_df.iloc[ind+1]['Street Name'] and ind != route_df.index[-1]:
            pass
        else:
            turn_indexes.append(ind)
    # Use the above function to get the direction of these turns 
    turns_to_make = dict()
    for ind in turn_indexes:
        turn_dir = get_turn_dir(graph, route_df.iloc[ind]['Start Node'], route_df.iloc[ind]['End Node'], route_df.iloc[ind+1]['End Node'])
        turns_to_make[ind] = turn_dir
    # Create a new column in the dataframe for this information
    turns_list = [None] * len(route_df)
    for i in turns_to_make.keys():
        turns_list[i] = turns_to_make[i]
    
    route_df['Turn Direction'] = turns_list
    return route_df

In [41]:
route_df = add_turns(full_route_df, G)
route_df

,Start Node,Street Name,Length,Direction,Speed,End Node,Destination?,Turn Direction
0,392180675,Dundas Street East,53.956,east,50.0,392176655,No,None
1,392176655,Dundas Street East,48.091,east,50.0,392170942,No,None
2,392170942,Dundas Street East,122.209,east,50.0,392175479,No,None
3,392175479,Dundas Street East,96.111,east,50.0,392169757,No,None
4,392169757,Dundas Street East,42.366,east,50.0,287482863,No,None
...,...,...,...,...,...,...,...,...
63,3877460870,Thickson Road,98.485,north,55.2,1077631215,No,None
64,1077631215,Thickson Road,28.271,north,55.2,3877460873,No,None
65,3877460873,Thickson Road,12.117,north,55.2,3877460874,No,None
66,3877460874,Thickson Road,29.791,north,55.2,3877460875,No,None


In [42]:
def avoid_a_street(graph, street_list):
    """
    Will take a list of street names and return a new graph with these
    edges removed.
    Since the graph is defined at the start and will be used for all uses,
    we first make a copy of the graph before manipulating it.
    Returns a new graph object with appropriate edges removed.
    """
    G2 = nx.Graph.copy(graph)
    edges_to_remove = []
    for edge in G2.edges:
        try:
            # Get the street name for each edge segment
            seg = G2.edges[edge][0]['name']
            # Add to list of edges to be removed
            if seg in street_list:
                edges_to_remove.append(edge)
        except KeyError:
            pass
    # Remove edges
    G2.remove_edges_from(edges_to_remove)
    return G2

In [43]:
def avoid_a_roadtype(graph, street_type_list):
    """
    Will take a road type - 'motorway', 'secondary', 'tertiary', 'residential' - 
    and remove this edges from the graph. 
    This will allow a user to avoid high speed, or low speed, roads, if possible.
    
    NOTE:
    This function will always follow avoid_a_street and so will take a copy of a graph.
    Graph copies have their information stored at inside a nested dictionary.
    """
    G2 = nx.Graph.copy(graph)
    edges_to_remove = []
    for edge in G2.edges:
        try:
            # The highway attribute of the edge gives the road type
            seg = G2.edges[edge][0][0]['highway']
            if seg in street_type_list:
                edges_to_remove.append(edge)
        except KeyError:
            pass
    G2.remove_edges_from(edges_to_remove)
    return G2

### Getting User Specifications

In [44]:
def get_starting_loc():
    """
    Prompts the user for their starting address.
    """
    print('Enter your starting address and city in full.')
    print('Ex: 23 Vanier Street Whitby')
    start = input()
    return start.title()

In [45]:
def get_destinations():
    """
    Prompts the user for their destinations.
    """
    print('\nEnter your destination addresses and cities in full.')
    print('Press enter after each entry.')
    print('Type "DONE" when you are finished.')
    destinations = []
    while True:
        dest = input()
        if dest.upper() == 'DONE':
            break
        elif dest:
            destinations.append(dest.title())
    return destinations

In [56]:
def get_customizations():
    """
    Asks the user to enter a list of streets they want to avoid,
    one at a time.
    Returns a list of street names.
    """
    print('\nEnter any streets you want to avoid.')
    print('Enter in full. ex: "Brock Street".')
    print('Press enter after each entry.')
    print('Type "DONE" when you are finished.')
    avoid = []
    while True:
        street = input()
        if street.upper() == 'DONE':
            break
        else:
            avoid.append(street.title())
    return avoid

In [47]:
def get_customizations2():
    """
    Gets further user specifications based on road speeds.
    Returns a list of road types to be removed.
    """
    avoid = []
    highways = input('Avoid highways? (Y/N): ')
    if highways.lower().startswith('y'):
        avoid.append('motorway')

    return avoid

In [48]:
def get_customizations3():
    """
    Gets further user specifications on what to optimize on.
    """
    how_opt = input('Do you want to optimize based on travel time (T) or distance traveled (D)? ')
    if how_opt.lower().startswith('T'):
        return 'travel_time'
    else:
        return 'length'

### Putting it Together

In [44]:
# Build the graph - this is Whitby/Oshawa for testing purposes - east, west, south, north
G = build_graph(43.984503, 43.862879, -78.954552, -78.821660)

In [58]:
def run_multi_opt_path():
    """
    This is the main function that takes all user inputs and returns the optimized routes 
    for each of the drivers. 
    """
    # Get the user's starting location and destination
    # If their input does not provide coordinates - if they enter a spelling of a street
    # that does not exist - the address_to_coord function will alert them and here they will
    # be reprompted for a valud address. 
    while True:
        start = get_starting_loc()
        start = address_to_coord(start)
        if start:
            break
    
    # Get destination addresses 
    dest = get_destinations()
    # Get the coordinates of each
    dest_coords = []
    for i in dest:
        dest_coords.append(address_to_coord(i))
    
    # Prompt user for the number of drivers    
    num_drivers = int(input('How many drivers are available? (Ex: 2) '))
    
    # Cluster the destinations according to this number
    clusters = cluster_destinations(dest_coords, num_drivers)
    
    # Get the user's customizations 
    roads_to_avoid = get_customizations()
    speeds_to_avoid = get_customizations2()
    opt_type = get_customizations3()
    
    # Use the functions to remove necessary edges from the graph
    G2 = avoid_a_street(G, roads_to_avoid)
    G3 = avoid_a_roadtype(G2, speeds_to_avoid)
    
    # Convert locations into nodes
    start_node = coords_to_node(G2, start)
    # Get destination nodes
    destination_nodes = []
    for i in range(len(clusters)):
        inodes = []
        for x in clusters[i]:
            inodes.append(coords_to_node(G, x))
        destination_nodes.append(inodes)
    
    # For each driver, get their route
    for driver in range(len(clusters)):
        # Display the driver number and their assigned destinations 
        print('\nDriver', driver+1, 'Route:')
        print('Destinations: ', destination_nodes[driver])
        # Create empty list of their visited destinations and their overall route
        visited = []
        route = []
        # Use the algorithm to build these lists
        try:
            aStarMulti(start_node, destination_nodes[driver], G3, opt_type, visited, route)
        except:
            return 'There is no possible path with these specifications.'
        # Convert the route into one list
        route_list = []
        route_list.append(route[0][0])
        for i in route:
            for j in i:
                if route_list[-1] != j:
                    route_list.append(j)
        # Get the full route information
        full_route_df = full_route_info(G, route_list, visited)
        # Add in the turns
        route_df = add_turns(full_route_df, G)
        # Display the route information as a dataframe
        # MORE WORK WILL BE DONE HERE TO SIMPLY THIS OUTPUT
        # (ie. combining segments that follow the same road)
        print(route_df)

### TESTING

SCENARIO:  Canada Post has a depot in Whitby, along with six post office locations.  We have a certain number of trucks that will take sorted mail from the depot to the various locations for delivery in their individual areas.

![Post Offices Whitby](canada_post.png)

In [62]:
# Testing the above scenario
# Remove some of Whitby's most common roads

In [51]:
run_multi_opt_path()

Enter your starting address and city in full.
Ex: 23 Vanier Street Whitby
209 Dundas St E Whitby

Enter your destination addresses and cities in full.
Press enter after each entry.
Type "DONE" when you are finished.
910 Dundas St W Whitby
3100 Garden Street Whitby
4081 Thickson Road N Whitby
728 Anderson Street Whitby
1801 Dundas St E Whitby
3500 Brock St N Whitby
done
How many drivers are available? (Ex: 2) 3

Enter any streets you want to avoid.
ex: "Brock Street".
Press enter after each entry.
Type "DONE" when you are finished.
Rossland Road East
Rossland Road West
Taunton Road East
Taunton Road West
done
Avoid highways? (Y/N): n
Do you want to optimize based on travel time (T) or distance traveled (D)? d

Driver 1 Route:
Destinations:  [392171046, 290584416]
    Start Node         Street Name   Length  Direction  Speed    End Node  \
0    392180675        Perry Street  214.096  northwest   46.5   392173691   
1    392173691        Perry Street  144.853      north   46.5   392175584

In [63]:
# Create a situation where no path will be possible

In [59]:
run_multi_opt_path()

Enter your starting address and city in full.
Ex: 23 Vanier Street Whitby
209 Dundas St E Whitby

Enter your destination addresses and cities in full.
Press enter after each entry.
Type "DONE" when you are finished.
4081 Thickson Road N Whitby
done
How many drivers are available? (Ex: 2) 1

Enter any streets you want to avoid.
Enter in full. ex: "Brock Street".
Press enter after each entry.
Type "DONE" when you are finished.
Dundas Street East
Perry Street
done
Avoid highways? (Y/N): n
Do you want to optimize based on travel time (T) or distance traveled (D)? d

Driver 1 Route:
Destinations:  [3877460879]


'There is no possible path with these specifications.'